In [2]:

#Instaliuojame reikalingas bibliotekas
%pip install datasets
%pip install -U transformers
%pip install peft
!pip install torch
!pip install trl


!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# For quantization(kvantizavimui)
!pip install bitsandbytes

# Modulio vertinimui
!pip install rouge-score



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 15.6 MB/s eta 0:00:00
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━

In [14]:

import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments,DataCollatorWithPadding, BitsAndBytesConfig
from datasets import Dataset
import torch
from huggingface_hub import login
import pandas as pd
from huggingface_hub import login
from trl import SFTConfig, SFTTrainer
from pathlib import Path
from peft import get_peft_model, LoraConfig
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score
from transformers import EarlyStoppingCallback

import numpy as np

login()
# 1: Load your dataset
# data = pd.read_csv('structured_conversation_data.csv')  # Update with your actual dataset path


In [27]:
# 1. Nustatome LoRa nustatymus.
# 1: Nustatome LoRa konfigūraciją.
lora_config_4 = LoraConfig(
    r=8,  # Mažo rango faktorius
    lora_alpha=16,  # Skalės faktorius LoRa atnaujinimams
    lora_dropout=0.1,  # Dropout norma LoRa
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"  # Užduoties tipas, skirta priežastiniam kalbos modeliavimui
)
# EOS_TOKEN = tokenizer.eos_token # Reikia pridėti, kad nesikartotų tas pats atsakymas
def formatting_prompts_func(row):
      input_text = row["Input"]
      output_text = row["Output"]
      text = sablonas.format(input_text, output_text, truncation=True) + EOS_TOKEN
      return text

for i in range(1):
  # 2. Inicijuojame modelį
  bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Įjungiame 4-bit kvantizavimą
    bnb_4bit_compute_dtype="bfloat16",  # Nustatome skaičiavimo dtype, pvz., bfloat16
    bnb_4bit_use_double_quant=True,  # Pasirinkimas dvigubai kvantizuoti (jei palaikoma)
)
  checkpoint = 'google/gemma-2-2b-it'

  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,  # Pateikiame konfigūraciją
    device_map="auto"  # Automatiškai priskiria modelį prie pasiekiamų įrenginių
)

  tokenizer = AutoTokenizer.from_pretrained(checkpoint)

  lora_config = lora_config_4

  #3. Pridedame LoRa prie modelio
  model = get_peft_model(model, lora_config)

  # 4. išaukiam formatavimo funkcija.
  sablonas = """ Žemiau pateikiama instrukcija ir atsakymai:

  ### Klausimas:
  {}

  ### Atsakymas:
  {}"""

  EOS_TOKEN = tokenizer.eos_token # Reikia pridėti, kad nesikartotų tas pats atsakymas

  dataset_df = pd.read_csv("structured_conversation_data_su_test.csv")
  dataset_df['text'] = dataset_df.apply(formatting_prompts_func, axis=1) # Formatuojame eilutes ir pridedame naują stulpelį "text", kuris vėliau bus naudojamas Trainer
  
  #5. Padaliname duomenis į treniravimo ir vertinimo rinkinius (pvz., 80% treniravimui, 20% vertinimui)
  train_size = int(0.8 * len(dataset_df))
  train_df = dataset_df[:train_size]
  eval_df = dataset_df[train_size:]

  # Paverčiame DataFrame į Hugging Face Dataset
  train_dataset = Dataset.from_pandas(train_df)
  eval_dataset = Dataset.from_pandas(eval_df)
  #6. Mokymo argumentai, ir trenerivavimo nustatymai
  training_args = SFTConfig(
      output_dir='res',
      packing=False,
      report_to='none',
      optim="adamw_8bit",
      logging_steps=500,
      eval_strategy="steps",
      eval_steps=500,
      max_steps=10080,
      # gradient_checkpointing=True,  # Įjungiame gradientų tikrinimą, kad išsaugotume atmintį GPU, bet nelabai dirba su 4 bit quatizavimu.
      max_seq_length=256,
      per_device_train_batch_size=4,
      per_device_eval_batch_size=1,
      learning_rate = 2e-4,
    )

  trainer = SFTTrainer(
  model=model,
  args=training_args,
  tokenizer=tokenizer,
  train_dataset=train_dataset,
  eval_dataset=eval_dataset,
  dataset_text_field='text',  # Užtikriname, kad tekstas bus naudojamas kaip įvestis
  peft_config=lora_config,  # Naudojame atitinkamą LoRA konfigūraciją
  )

  # 7. Pradedame "Fine-Tuning" procesą.
  trainer.train()

  # 8. Išsaugome apmokytą modelį, bei kofiguraciją.
  modelio_kelias="/modelis_10080"
  model.save_pretrained(modelio_kelias)
  tokenizer.save_pretrained(modelio_kelias)
  duomenys = """
    LoRa nustatymai: {}
    Training argumentai: {}
  """
  formatas = duomenys.format(lora_config, training_args)

  # Modelio pavadinimas ir direktorijos nustatymas
  directory = Path(modelio_kelias)
  directory.mkdir(parents=True, exist_ok=True)  # Sukuria katalogą, jei jis neegzistuoja

  # Apibrėžiame failo kelią ir įrašome konfigūracijos tekstą
  file_path = directory / "konfiguracija.txt"
  with file_path.open("a", encoding="utf-8") as f:
      f.write("\n" + formatas)


  print("Treniruotė baigta! Modelis išsaugotas.")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/5754 [00:00<?, ? examples/s]

Map:   0%|          | 0/1439 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss


Treniruotė baigta! Modelis išsaugotas.


## Apmokyto modelio įkelimas, ar paleidimas.

### Įsikeliame modelį.

In [6]:
from google.colab import files
uploaded = files.upload()
import os
import zipfile


Saving modelis_[0].zip to modelis_[0].zip


### Išpakuojame modelį.

In [ ]:

# Nurodome failo kelią
zip_file_path = "./model_[0].zip"

# Nurodome kur įkelsime.
extract_to = "./"

# Išpakuojame
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Patvrtinimas
print(f"Files extracted to {extract_to}")
print(os.listdir(extract_to))

Toliau galima naudotis įprastai..., tiosog pakeičiant modelio pavadinimus(ptaths)

## Čiatas.

In [26]:
import torch
import json
import time
from transformers import AutoModelForCausalLM, AutoTokenizer

modelio_kelias="./modelis_2500_v2"
# Užkrauname išankstinio mokymo modelį ir žodyną
model = AutoModelForCausalLM.from_pretrained(modelio_kelias)
tokenizer = AutoTokenizer.from_pretrained(modelio_kelias)

# Užtikriname, kad modelis būtų tinkamoje įrangoje (GPU, jei pasiekiama, arba CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Alpaca stiliaus šablonas
sablonas = """ Žemiau yra pateikiamas klausimas ir atsakymas:

### Klausimas:
{}

### Atsakymas:
{}"""

# Nustatome pradinius modelio parametrus
temperature = 0.7  # Kontroliuojame atsako atsitiktinumą: 1.0 padaro atsakymą atsitiktinį, 0.1 - deterministinį
top_p = 0.9  # Nucleus Sampling: apribojame žodžių pasirinkimą pagal kumuliatyvią tikimybę, kurios vertė ≤ p
top_k = 50  # Top-k Sampling: ribojame žodžių pasirinkimą tik pagal top k labiausiai tikėtinas parinktis

max_new_tokens = 100  # Maksimalus naujų žodžių (tokenų) skaičius

# Pabandome užkrauti pokalbio istoriją iš JSON failo, jei jis egzistuoja
istorjos_failas = "pokalbio_istorija.json"
try:
    with open(istorjos_failas, "r") as file:
        pokalbio_istorija = json.load(file)
except FileNotFoundError:
    pokalbio_istorija = []

# Funkcija pradėti pokalbį ir išmatuoti atsakymo laiką
def pradėti_pokalbį():
    global temperature, top_p, top_k

    while True:
        vartotojo_įvestis = input("Klausimas: ").strip()
        if vartotojo_įvestis.lower() == "exit":
            print("Išeiname iš pokalbio.")
            break

        # Užfiksuojame pradžios laiką
        start_time = time.time()

        # Paruošiame įvestį modeliui (paliekame išvestį tuščią generavimui)
        inputs = tokenizer(
            [
                sablonas.format(
                    vartotojo_įvestis,  # Vartotojo klausimas
                    ""  # Paliekame išvestį tuščią modeliui generuoti atsakymą
                )
            ],
            return_tensors="pt"
        ).to(device)  # Užtikriname, kad įvestys būtų tame pačiame įrenginyje kaip ir modelis

        # Generuojame modelio atsakymą su atnaujintais parametrais
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,  # Naudojame atnaujintą temperature parametrą
            top_p=top_p,  # Naudojame atnaujintą top_p parametrą
            top_k=top_k  # Naudojame top_k parametrą
        )

        # Užfiksuojame pabaigos laiką
        end_time = time.time()

        # Apskaičiuojame laiką, kurį prireikė modeliui atsakyti
        elapsed_time = end_time - start_time
        print(f"Modelio atsakymo laikas: {elapsed_time:.4f} sekundės")  # Išvedame atsakymo laiką sekundėmis

        # Dekoduojame ir išvedame modelio atsakymą
        response = tokenizer.decode(output[0], skip_special_tokens=True)

        # Išvedame tik asistento atsakymą (neatsikartojame vartotojo įvesties)
        asistentas_response = response.split("### Atsakymas:")[-1].strip()  # Ištraukiame atsakymą po '### Atsakymas:'
        print("Asistentas:", asistentas_response)

        # Saugojame pokalbio istoriją į JSON failą po kiekvieno atsakymo
        pokalbio_istorija.append({"vartotojas": vartotojo_įvestis, "asistentas": asistentas_response})
        with open(istorjos_failas, "w") as file:
            json.dump(pokalbio_istorija, file, ensure_ascii=False, indent=4)

# Pradėkite pokalbį
pradėti_pokalbį()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Klausimas: Sveikas


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Modelio atsakymo laikas: 0.8441 sekundės
Asistentas: Labas, kaip tau sekasi?
Klausimas: Gerai o tau ?
Modelio atsakymo laikas: 8.5981 sekundės
Asistentas: Labas, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne, ne
Klausimas: Kiek metu tau ?
Modelio atsakymo laikas: 8.6013 sekundės
Asistentas: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
Klausimas: kur gyveni ?
Modelio atsakymo laikas: 1.4030 sekundės
Asistentas: Labas, nezinau, bet galvoju, kad Kaunas


KeyboardInterrupt: Interrupted by user

## Pasaulio žinios. Klausimai patikrinti, ar nepablogėjo bendros žinios.

In [22]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import bitsandbytes as bnb

# Užtikriname, kad modelis būtų tinkamoje įrangoje
sablonas = """ Žemiau yra pateikiamas klausimas ir atsakymas:

### Klausimas:
{}

### Atsakymas:
{}"""

# Užkrauname duomenis iš CSV failo
klausimai = pd.read_csv("pasaulio_zinios.csv")
istorija = []  # Saugojame pokalbių istoriją

# Nustatome modelio kelią
modelio_kelias = './modelis_2500'  # Keiskite kelią, jei norite naudoti kitą modelį

# Užkrauname žodyną
tokenizer = AutoTokenizer.from_pretrained(modelio_kelias)

# Užkrauname modelį 4 bitų tikslumu
model = AutoModelForCausalLM.from_pretrained(
    modelio_kelias,
    load_in_4bit=True,
    device_map="auto",
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Sukuriame ciklą per klausimus iš duomenų
for i in range(len(klausimai)):
    # Paimame įvestį iš duomenų
    user_input = klausimai["Input"][i]

    # Jei įvedama "exit", išeiname iš pokalbio
    if user_input.lower() == "exit":
        print("Išeiname iš pokalbio.")
        break

    # Paruošiame įvestį modeliui
    inputs = tokenizer(
        [
            sablonas.format(
                user_input,  # Vartotojo klausimas
                ""  # Paliekame išvestį tuščią, kad modelis sugeneruotų atsakymą
            )
        ],
        return_tensors="pt"
    ).to(device)  # Užtikriname, kad įvestys būtų įrašytos į teisingą įrenginį

    # Generuojame modelio atsakymą
    output = model.generate(**inputs, max_new_tokens=30)

    # Dekoduojame atsakymą
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    assistant_response = response.split("### Atsakymas:")[-1].strip()

    # Saugojame pokalbio istoriją ir pridedame modelio pavadinimą
    istorija.append({
        "Klausimas": user_input,
        "Asistentas": assistant_response,
        "Tikrasis": klausimai["Output"][i],
        "Modelis": modelio_kelias  # Pridedame modelio pavadinimą kaip naują stulpelį
    })

# Išsaugome pokalbių istoriją į CSV failą
df = pd.DataFrame(istorija)
df.to_csv("pasaulio_zinios_istorija.csv", index=False)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Rough score naudojamas kad įvertinti, kaip apsimokė modelis

In [23]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import bitsandbytes as bnb
from rouge_score import rouge_scorer
from transformers import BitsAndBytesConfig

# # Užtikrinkite, kad modelis būtų teisingame įrenginyje
sablonas = """ Žemiau yra pateikiamas klausimas ir atsakymas:

### Klausimas:
{}

### Atsakymas:
{}"""

# Nuskaityti duomenis iš CSV
klausimai = pd.read_csv("25_N.klausimu_po_apmokymo_messanger.csv")
rough_score = []  # Saugo pokalbio istoriją
modelio_kelias='./modelis_2500'
# Nuskaityti žodyną
tokenizer = AutoTokenizer.from_pretrained(modelio_kelias)

# Sukurti BitsAndBytesConfig, kad įkeltumėte modelį 4 bitų tikslumu
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Nuskaityti modelį su konfigūruotu bitsandbytes
model = AutoModelForCausalLM.from_pretrained(
    modelio_kelias,
    quantization_config=bnb_config,  # Naudokite BitsAndBytes konfigūraciją
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# For loop to process each row in the DataFrame
for i in range(len(klausimai)):
    user_input = klausimai["Input"][i]

    if user_input.lower() == "exit":
        print("Išeinama iš pokalbio.")
        break

    # Paruošti įvestį modeliui
    inputs = tokenizer(
        [
            sablonas.format(
                user_input,  # Klausimo įvestis
                ""  # Palikite atsakymą tuščią modeliui generuoti
            )
        ],
        return_tensors="pt"
    ).to(device)

    # Sugeneruoti modelio atsakymą
    output = model.generate(**inputs, max_new_tokens=30)

    # Iššifruoti atsakymą
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    assistant_response = response.split("### Atsakymas:")[-1].strip()

    # Inicijuoti ROUGE įvertintoją
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

    # Apskaičiuoti ROUGE įverčius
    scores = scorer.score(klausimai["Output"][i], assistant_response)

    # Išsaugoti pokalbio istoriją
    rough_score.append({
        "Modelis": modelio_kelias,
        "Klausimas": user_input,
        "Asistentas": assistant_response,
        "Tikrasis": klausimai["Output"][i],
        "Rezultatai": [scores['rouge1'],
                       # scores['rouge2'],
                       # scores['rougeL']
                       ]
    })

# Išsaugoti ROUGE įverčius į CSV failą
df = pd.DataFrame(rough_score)
df.to_csv("rough_score.csv")


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Gali prireikti atnaujinti bibletiokas, jeig slideriai neveiks.

In [ ]:
# output = model.generate(
#     inputs,
#     max_length=20,                # Atsakymas bus apribotas iki 20 žodžių. Kontroliuoja atsakymo ilgį.
#     temperature=1.0,              # 1.0 reikšmė reiškia vidutinį atsitiktinumą renkantis žodžius.
#     top_k=50,                     # Apsvarstomi tik 50 tikėtiniausių žodžių, ribojama įvairovė.
#     top_p=1.0,                    # Naudojami visi prieinami žodžiai mėginimui, be apribojimų.
#     do_sample=False,              # Be atsitiktinumo; visada pasirenkamas tikriausias žodis kiekviename etape.
#     num_beams=1,                  # Vienas spindulio paieška (nenaudojama spindulių paieška, kuriama tik viena sekos versija).
#     repetition_penalty=1.0,       # Nėra bausmės už pasikartojimą; leidžiama kartoti žodžius.
# )


### Taik kaip pagalbinė priemonė, kad išstrinti folders, jeigu kas nepavyksta...

In [ ]:
# import os
# import shutil

# # Path to the folder you want to delete
# # folder_path = 'logs'

# # Delete the folder and its contents
# # shutil.rmtree('res')

# listas=["modelis_[0]", "modelis_[1]","modelis_[2]","modelis_[3]","modelis_[4]","modelis_[5]","res"]
# for item in listas:
#   shutil.rmtree(item)

### Pagalbinė priemonė, jeigu reikia unzipinti modelį. Arba parsisiūsti...

In [ ]:

  # import os


  # # listas=["finetuned_gemma_lora", "results", 'wandb',"logs" ]
  # # for item in listas:
  # #   shutil.rmtree(item)

  # !zip -r modelis_[0].zip modelis_[0]
  # !zip -r modelis_1440.zip modelis_1440

  # from google.colab import files
  # # files.download('modelis_[0].zip')
  # files.download('modelis_1440.zip')


  # import zipfile

  # # Path to the ZIP file you uploaded
  # zip_file_path = "finetuned_gemma_lora.zip"  # Replace with your actual file path

  # # Directory where you want to extract the files (you can change the destination directory)
  # extract_path = "./"

  # # Unzip the file
  # with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
  #     zip_ref.extractall(extract_path)

  # print(f"Model files extracted to {extract_path}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>